# Consolidating the Data

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('Data/clickbait_data.csv')

In [3]:
data

,headline,clickbait
0,Should I Get Bings,1
1,Which TV Female Friend Group Do You Belong In,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1
3,"This Vine Of New York On ""Celebrity Big Brothe...",1
4,A Couple Did A Stunning Photo Shoot With Their...,1
...,...,...
31995,"To Make Female Hearts Flutter in Iraq, Throw a...",0
31996,"British Liberal Democrat Patsy Calton, 56, die...",0
31997,Drone smartphone app to help heart attack vict...,0
31998,"Netanyahu Urges Pope Benedict, in Israel, to D...",0


# Pre-processing Text

In [6]:
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/taejoon/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/taejoon/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /Users/taejoon/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [8]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'[a-zA-Z0-9]+')

In [15]:
tokenized_data = data
for i in range(len(data)):
    tokenized_data['headline'][i] = tokenizer.tokenize(data['headline'][i])

/Users/taejoon/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
tokenized_data

,headline,clickbait
0,"[should, i, get, bings]",1
1,"[which, tv, female, friend, group, do, you, be...",1
2,"[the, new, star, wars, the, force, awakens, tr...",1
3,"[this, vine, of, new, york, on, celebrity, big...",1
4,"[a, couple, did, a, stunning, photo, shoot, wi...",1
...,...,...
31995,"[to, make, female, hearts, flutter, in, iraq, ...",0
31996,"[british, liberal, democrat, patsy, calton, 56...",0
31997,"[drone, smartphone, app, to, help, heart, atta...",0
31998,"[netanyahu, urges, pope, benedict, in, israel,...",0


In [18]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))

In [20]:
filtered_data=[]
for w in tokenized_data:
    if w not in stop_words:
        filtered_data.append(w)

In [21]:
filtered_data

['headline', 'clickbait']

In [ ]:
list(tokenized_data.loc[tokenized_data['clickbait'] == 1]['headline'])

In [58]:
flat_clickbait = [item for sublist in list(tokenized_data.loc[tokenized_data['clickbait'] == 1]['headline']) for item in sublist]

In [60]:
len(flat_clickbait)

164293

In [56]:
[headline for headline in list(tokenized_data.loc[tokenized_data['clickbait'] == 1]['headline'])]

[['should', 'i', 'get', 'bings'],
 ['which', 'tv', 'female', 'friend', 'group', 'do', 'you', 'belong', 'in'],
 ['the',
  'new',
  'star',
  'wars',
  'the',
  'force',
  'awakens',
  'trailer',
  'is',
  'here',
  'to',
  'give',
  'you',
  'chills'],
 ['this',
  'vine',
  'of',
  'new',
  'york',
  'on',
  'celebrity',
  'big',
  'brother',
  'is',
  'fucking',
  'perfect'],
 ['a',
  'couple',
  'did',
  'a',
  'stunning',
  'photo',
  'shoot',
  'with',
  'their',
  'baby',
  'after',
  'learning',
  'she',
  'had',
  'an',
  'inoperable',
  'brain',
  'tumor'],
 ['how',
  'to',
  'flirt',
  'with',
  'queer',
  'girls',
  'without',
  'making',
  'a',
  'total',
  'fool',
  'of',
  'yourself'],
 ['32',
  'cute',
  'things',
  'to',
  'distract',
  'from',
  'your',
  'awkward',
  'thanksgiving'],
 ['if', 'disney', 'princesses', 'were', 'from', 'florida'],
 ['what',
  's',
  'a',
  'quote',
  'or',
  'lyric',
  'that',
  'best',
  'describes',
  'your',
  'depression'],
 ['natalie',


In [ ]:
flat_list = [item for sublist in regular_list for item in sublist]
flat_cb = [item for sublist in cb_list for item in sublist]

In [52]:
list(tokenized_data.loc[tokenized_data['clickbait'] == 1]['headline'])

[['should', 'i', 'get', 'bings'],
 ['which', 'tv', 'female', 'friend', 'group', 'do', 'you', 'belong', 'in'],
 ['the',
  'new',
  'star',
  'wars',
  'the',
  'force',
  'awakens',
  'trailer',
  'is',
  'here',
  'to',
  'give',
  'you',
  'chills'],
 ['this',
  'vine',
  'of',
  'new',
  'york',
  'on',
  'celebrity',
  'big',
  'brother',
  'is',
  'fucking',
  'perfect'],
 ['a',
  'couple',
  'did',
  'a',
  'stunning',
  'photo',
  'shoot',
  'with',
  'their',
  'baby',
  'after',
  'learning',
  'she',
  'had',
  'an',
  'inoperable',
  'brain',
  'tumor'],
 ['how',
  'to',
  'flirt',
  'with',
  'queer',
  'girls',
  'without',
  'making',
  'a',
  'total',
  'fool',
  'of',
  'yourself'],
 ['32',
  'cute',
  'things',
  'to',
  'distract',
  'from',
  'your',
  'awkward',
  'thanksgiving'],
 ['if', 'disney', 'princesses', 'were', 'from', 'florida'],
 ['what',
  's',
  'a',
  'quote',
  'or',
  'lyric',
  'that',
  'best',
  'describes',
  'your',
  'depression'],
 ['natalie',


# Feature Engineering

In [4]:
data['headline'] = data['headline'].apply(lambda x: x.lower())

In [5]:
data

,headline,clickbait
0,should i get bings,1
1,which tv female friend group do you belong in,1
2,"the new ""star wars: the force awakens"" trailer...",1
3,"this vine of new york on ""celebrity big brothe...",1
4,a couple did a stunning photo shoot with their...,1
...,...,...
31995,"to make female hearts flutter in iraq, throw a...",0
31996,"british liberal democrat patsy calton, 56, die...",0
31997,drone smartphone app to help heart attack vict...,0
31998,"netanyahu urges pope benedict, in israel, to d...",0
